In [39]:
import json
from plotly.graph_objs import Scattergeo, Layout
from plotly import offline


confirmed_list = []
lons = []
lats = []
texts = []
province = []


# Lire le fichier json
with open('csvjson.json','r') as file:
    all_data = json.loads(file.read())
    
for x in all_data:
    confirmed = x['3/25/20']
    lon = x['Long']
    lat = x['Lat']
    title = x['Country/Region']
    provinces = x['Province/State']
    
    confirmed_list.append(confirmed)
    lons.append(lon)
    lats.append(lat)
    texts.append(provinces+ '\n' + title + '\n' + str(confirmed))
    
# Préparation du graphique
graph_data = [{
    'type': 'scattergeo',
    'lon': lons,
    'lat': lats,
    'text': texts,
    'marker': {
        #'size': [0.005*abs(confirmed) for confirmed in confirmed_list], # la taille des bulles
        'size': 4*confirmed,
        'color': confirmed_list,
        'colorscale': 'Cividis', # autre options 'RdBu', 'Cividis'
        'colorbar': {'title': 'Nombre de cas'},
    },
}]
# Titre du graphique
graph_layout = Layout(title='COVID-19')

# Génération du graphique, comme fichier externe 'covid.html'
fig = {'data': graph_data, 'layout': graph_layout}

offline.plot(fig, filename='covid.html')

'covid.html'

In [31]:
import requests 
import json 
import pandas 
from datetime import date
from datetime import timedelta
pandas.options.display.float_format = '{:.2f}'.format

today = date.today()
hier = today - timedelta(days=1)
hier = hier.strftime("%#m/%#d/%y")
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

#try:
   # r = requests.get(url)
   # r.raise_for_status()
#except requests.exceptions.RequestException as erreur:
#    print("Erreur de connexion à l'adresse web suivante `{0}` : {1}".format(url, erreur))
#else: 
real_data = pandas.read_csv(url).fillna('N.D')
data = real_data.groupby(['Country/Region', 'Province/State'])[[hier]].sum()
data_sorted = data.sort_values(by=hier, ascending = False)
data_sorted.head(20)

,,4/1/20
Country/Region,Province/State,
US,N.D,213372
Italy,N.D,110574
Spain,N.D,104118
Germany,N.D,77872
China,Hubei,67802
France,N.D,56989
Iran,N.D,47593
United Kingdom,N.D,29474
Switzerland,N.D,17768


In [44]:
import requests
from contextlib import closing
import csv
from plotly.graph_objs import Scattergeo, Layout
from plotly import offline

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_list = []
lons = []
lats = []
texts = []


with closing(requests.get(url, stream=True)) as r:
    f = (line.decode('utf-8') for line in r.iter_lines())
    reader = csv.reader(f, delimiter=',', quotechar='"')
    for x in reader:
        confirmed = x[-1]
        lon = x[3]
        lat = x[2]
        title = x[1]
        provinces = x[0]

        confirmed_list.append(confirmed)
        lons.append(lon)
        lats.append(lat)
        texts.append(provinces+ '\n' + title + '\n' + str(confirmed))
        del confirmed_list[0]
# Préparation du graphique
graph_data = [{
    'type': 'scattergeo',
    'lon': lons,
    'lat': lats,
    'text': texts,
    'marker': {
        'size': 4*int(confirmed),
        'color': confirmed_list,
        'colorscale': 'Cividis', # autre options 'RdBu', 'Cividis'
        'colorbar': {'title': 'Nombre de cas'},
    },
}]
# Titre du graphique
graph_layout = Layout(title='COVID-19')

# Génération du graphique, comme fichier externe 'covid.html'
fig = {'data': graph_data, 'layout': graph_layout}

offline.plot(fig, filename='covid_csv.html')

'covid_csv.html'